# Decision Trees e Random Forests

In [7]:
import findspark
findspark.init('/home/macaubas/spark-3.2.1-bin-hadoop3.2')
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tree').getOrCreate()

22/05/19 15:25:16 WARN Utils: Your hostname, macaubas-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/05/19 15:25:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/19 15:25:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Imports

In [28]:
from pyspark.ml import Pipeline

from pyspark.ml.classification import (RandomForestClassifier, GBTClassifier, DecisionTreeClassifier)

from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

from pyspark.ml.feature import (VectorAssembler, StringIndexer)

## Carregandos dados

In [8]:
data = spark.read.csv('College.csv', inferSchema=True, header=True)

data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [10]:
for e in data.head(5):
    print(e)
    print('\n')

Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)


Row(School='Adelphi University', Private='Yes', Apps=2186, Accept=1924, Enroll=512, Top10perc=16, Top25perc=29, F_Undergrad=2683, P_Undergrad=1227, Outstate=12280, Room_Board=6450, Books=750, Personal=1500, PhD=29, Terminal=30, S_F_Ratio=12.2, perc_alumni=16, Expend=10527, Grad_Rate=56)


Row(School='Adrian College', Private='Yes', Apps=1428, Accept=1097, Enroll=336, Top10perc=22, Top25perc=50, F_Undergrad=1036, P_Undergrad=99, Outstate=11250, Room_Board=3750, Books=400, Personal=1165, PhD=53, Terminal=66, S_F_Ratio=12.9, perc_alumni=30, Expend=8735, Grad_Rate=54)


Row(School='Agnes Scott College', Private='Yes', Apps=417, Accept=349, Enroll=137, Top10perc=60, Top25perc=89, F_Undergrad=510, P_

## Formatando os dados

In [12]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [13]:
# Selecionando características de interesse
assembler = VectorAssembler(inputCols = ['Apps',
                                         'Accept',
                                         'Enroll',
                                         'Top10perc',
                                         'Top25perc',
                                         'F_Undergrad',
                                         'P_Undergrad',
                                         'Outstate',
                                         'Room_Board',
                                         'Books',
                                         'Personal',
                                         'PhD',
                                         'Terminal',
                                         'S_F_Ratio',
                                         'perc_alumni',
                                         'Expend',
                                         'Grad_Rate'],
                           outputCol='features')

In [14]:
output = assembler.transform(data)

In [15]:
output.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)



## Stringer Indexer

In [17]:
indexer = StringIndexer(inputCol = "Private", outputCol='Private_Index')

In [18]:
# Adicionando coluna indexada
output_fixed = indexer.fit(output).transform(output)

In [19]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- Private_Index: double (nullable = false)



In [21]:
final_data = output_fixed.select('features', 'Private_Index')

## Preparando dataset

In [22]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

## Modelos

In [55]:
# Criando modelos
dtc = DecisionTreeClassifier(labelCol='Private_Index', featuresCol='features')
rfc_1 = RandomForestClassifier(labelCol='Private_Index', featuresCol='features', numTrees=25)
rfc_2 = RandomForestClassifier(labelCol='Private_Index', featuresCol='features', numTrees=100)
rfc_3 = RandomForestClassifier(labelCol='Private_Index', featuresCol='features', numTrees=150)
gbt_1 = GBTClassifier(labelCol='Private_Index', featuresCol='features', maxDepth=5)
gbt_2 = GBTClassifier(labelCol='Private_Index', featuresCol='features', maxDepth=10)
gbt_3 = GBTClassifier(labelCol='Private_Index', featuresCol='features', maxDepth=15)

In [56]:
# Fitting model
dtc_model = dtc.fit(train_data)
rfc_1_model = rfc_1.fit(train_data)
rfc_2_model = rfc_2.fit(train_data)
rfc_3_model = rfc_3.fit(train_data)
gbt_1_model = gbt_1.fit(train_data)
gbt_2_model = gbt_2.fit(train_data)
gbt_3_model = gbt_3.fit(train_data)

In [57]:
# Previsões
dtc_preds = dtc_model.transform(test_data)
rfc_1_preds = rfc_1_model.transform(test_data)
rfc_2_preds = rfc_2_model.transform(test_data)
rfc_3_preds = rfc_3_model.transform(test_data)
gbt_1_preds = gbt_1_model.transform(test_data)
gbt_2_preds = gbt_2_model.transform(test_data)
gbt_3_preds = gbt_3_model.transform(test_data)

## Avaliando

### Binary Classification Evaluator

In [58]:
my_binary_eval = BinaryClassificationEvaluator(labelCol='Private_Index')

In [59]:
# DecisionTree e Random Forest possuem raw_prediction
print(f"DTC: {my_binary_eval.evaluate(dtc_preds):.3}")
print(f"RFC 25 Trees: {my_binary_eval.evaluate(rfc_1_preds):.3}")
print(f"RFC 100 Trees: {my_binary_eval.evaluate(rfc_2_preds):.3}")
print(f"RFC 150 Trees: {my_binary_eval.evaluate(rfc_3_preds):.3}")

DTC: 0.905
RFC 25 Trees: 0.982
RFC 100 Trees: 0.984
RFC 150 Trees: 0.984


In [60]:
# Ajustando evaluator rawPredictionCol
my_binary_eval2 = BinaryClassificationEvaluator(labelCol='Private_Index', rawPredictionCol='prediction')
print(f"GBT 5: {my_binary_eval2.evaluate(gbt_1_preds):.3}")
print(f"GBT 10: {my_binary_eval2.evaluate(gbt_2_preds):.3}")
print(f"GBT 15: {my_binary_eval2.evaluate(gbt_3_preds):.3}")

GBT 5: 0.886
GBT 10: 0.88
GBT 15: 0.88


Mudar a profundidade máxima do GBT parece não melhorar o desempenho do modelo. Uma possível razão pra um performance pior do que seus pares é uma má parametrização do modelo. É possível melhorar o modelo GBT.

### Multi Classification Evaluator

In [62]:
acc_eval = MulticlassClassificationEvaluator(labelCol='Private_Index', metricName='accuracy')

In [69]:
print(f"DTC: {acc_eval.evaluate(dtc_preds):.3}")
print(f"RFC 25 Trees: {acc_eval.evaluate(rfc_1_preds):.3}")
print(f"RFC 100 Trees: {acc_eval.evaluate(rfc_2_preds):.3}")
print(f"RFC 150 Trees: {acc_eval.evaluate(rfc_3_preds):.3}")
print(f"GTB 5: {acc_eval.evaluate(gbt_1_preds):.3}")
print(f"GTB 10: {acc_eval.evaluate(gbt_2_preds):.3}")
print(f"GTB 15: {acc_eval.evaluate(gbt_3_preds):.3}")

DTC: 0.917
RFC 25 Trees: 0.934
RFC 100 Trees: 0.947
RFC 150 Trees: 0.947
GTB 5: 0.917
GTB 10: 0.908
GTB 15: 0.908


Decision Tree e Gradient Burst Tree possuem precisões similares para configurações default. GBT perde performance ao aumentar a profundidade máxima. Por sua vez, os modelos de RandomForest para 100 e 150 árvores performaram igualmente e foram os modelos mais precisos.